## building an agent taht interacts with a search engine 
- ask agent questions 
- watch agent call the search tool 
- have conversations with the agent

[python.langchain tutorial](https://python.langchain.com/docs/tutorials/agents/)

In [ ]:
# import relevant functionality 
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

packages i need to install in virtual environment
```
pip install langchain
pip insall langgraph
```